In [1]:
from langchain_community.llms import LlamaCpp
from pprint import pprint
import pandas as pd
import numpy as np


In [2]:
n_gpu_layers = 1000  # Metal set to 1 is enough.
n_gpu_layers = 36  # Metal set to 1 is enough.
# n_gpu_layers = None  # Metal set to 1 is enough.
n_batch = 2048  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="models/llama-13b-hf_q8_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4060 Ti, compute capability 8.9, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from models/llama-13b-hf_q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length 

In [3]:
req = "Your task is to process and complete recipes that have missing data. You intelligently infer and fill in missing data based on complete examples. \
    A recipe has form of three columns, where at least one column has a value and the others can be masked. Columns:\n\nname: <NAME>\ningredients: <INGREDIENTS>\nsteps: <STEPS>\n\nContained \
    information:\n\n<NAME> is name of the recipe.\n<INGREDIENTS> consists of ingredients used in processing actions described in steps, separated by commas.\n<STEPS> are short sentences. \
    They begin with big letter and end with a dot. One step is one sentence. Steps explain what actions are needed to perform recipe in a correct manner.\n\n\
    Fill in the missing information in given recipe as accurately, logically, coherently, and truthfully as possible. It is necessary that the output is structured in the same way as shown in the examples above, where all columns contain values and the same structure.\
    This is a recipe to fill \n\nname: tilapia in mustard cream sauce\ningredients: tilapia fillets, fresh thyme, fresh ground black pepper, salt, cooking spray, chicken broth, portabella mushroom, whipping cream, dijon mustard\nsteps: <fill>"

req = "Your task is to process and complete recipes that have missing data. You intelligently infer and fill in missing data based on complete examples. \
    A recipe has form of three columns, where at least one column has a value and the others can be masked. Columns: name: <NAME> ingredients: <INGREDIENTS> steps: <STEPS> Contained \
    information: <NAME> is name of the recipe. <INGREDIENTS> consists of ingredients used in processing actions described in steps, separated by commas. <STEPS> are short sentences. \
    They begin with big letter and end with a dot. One step is one sentence. Steps explain what actions are needed to perform recipe in a correct manner. \
    Fill in the missing information in given recipe as accurately, logically, coherently, and truthfully as possible. It is necessary that the output is structured in the same way as shown in the examples above, where all columns contain values and the same structure.\
    This is a recipe to fill - name: tilapia in mustard cream sauce ingredients: tilapia fillets, fresh thyme, fresh ground black pepper, salt, cooking spray, chicken broth, portabella mushroom, whipping cream, dijon mustard. Please provide the steps filling for this recipe."

req = ""

In [4]:
df = pd.read_csv("train.csv")

In [5]:

[f"name: {e[2]} ; ingredients: {e[4]} ; preparation:" for e in [df.iloc[[1,5,8]].values[-1]]]
# [e for e in [df.iloc[[1,5,8]].values[-1]]]

['name: Tilapia in mustard cream sauce ; ingredients: tilapia fillets, fresh thyme, fresh ground black pepper, salt, cooking spray, chicken broth, portabella mushroom, whipping cream, dijon mustard ; preparation:']

In [22]:
def compose(gdf):
    comp = [f"name: {e[2]} ; ingredients: {e[4]} ; preparation:{e[3]}" for e in gdf.values[:-1]]
    # print(gdf.values[-1])
    comp = comp + [f"name: {e[2]} ; ingredients: {e[4]} ; preparation:" for e in [gdf.values[-1]]]
    return "; ".join(comp)

req = compose(df.iloc[[1,5,8]])
req = "The second planet in the solar system is "

In [23]:
req

'The second planet in the solar system is '

In [24]:
# res = llm.invoke("Simulate a rap battle between Stephen Colbert and John Oliver. Write only dialogue. Both of theme swear a lot and responses have to rhyme. They also fight on knives while rapping.")
res = llm.invoke(req)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     195.59 ms
llama_print_timings:      sample time =      45.72 ms /   256 runs   (    0.18 ms per token,  5599.67 tokens per second)
llama_print_timings: prompt eval time =     433.51 ms /     9 tokens (   48.17 ms per token,    20.76 tokens per second)
llama_print_timings:        eval time =   19503.41 ms /   255 runs   (   76.48 ms per token,    13.07 tokens per second)
llama_print_timings:       total time =   20347.49 ms /   264 tokens


In [25]:

print(res)

4.9 billion years old and, by most accounts, still has plenty of life left in it.
Mars’ surface looks like a cratered desert on Earth, but scientists believe there were once oceans and rivers there. There are signs of volcanic activity as well as canyons and valleys that suggest it was once habitable.
The planet’s atmosphere was likely thin and made mostly out of carbon dioxide, with water vapor mixed in to help keep it warm. As its orbit shifted over time, however, the temperature dropped until all life was extinct – though some argue that this could have been prevented if only we had been able to preserve our atmosphere better!
Nowadays Mars is cold enough to freeze carbon dioxide into dry ice; this substance will eventually turn into water vapor if exposed to enough sunlight or heat energy from volcanoes (which could help explain why some scientists think there may have been liquid water on Mars).
What would happen if you went to Mars?
If you went to Mars, you would be exposed to a 

In [26]:
res.split("name:")[0]

'4.9 billion years old and, by most accounts, still has plenty of life left in it.\nMars’ surface looks like a cratered desert on Earth, but scientists believe there were once oceans and rivers there. There are signs of volcanic activity as well as canyons and valleys that suggest it was once habitable.\nThe planet’s atmosphere was likely thin and made mostly out of carbon dioxide, with water vapor mixed in to help keep it warm. As its orbit shifted over time, however, the temperature dropped until all life was extinct – though some argue that this could have been prevented if only we had been able to preserve our atmosphere better!\nNowadays Mars is cold enough to freeze carbon dioxide into dry ice; this substance will eventually turn into water vapor if exposed to enough sunlight or heat energy from volcanoes (which could help explain why some scientists think there may have been liquid water on Mars).\nWhat would happen if you went to Mars?\nIf you went to Mars, you would be exposed

In [27]:
res

'4.9 billion years old and, by most accounts, still has plenty of life left in it.\nMars’ surface looks like a cratered desert on Earth, but scientists believe there were once oceans and rivers there. There are signs of volcanic activity as well as canyons and valleys that suggest it was once habitable.\nThe planet’s atmosphere was likely thin and made mostly out of carbon dioxide, with water vapor mixed in to help keep it warm. As its orbit shifted over time, however, the temperature dropped until all life was extinct – though some argue that this could have been prevented if only we had been able to preserve our atmosphere better!\nNowadays Mars is cold enough to freeze carbon dioxide into dry ice; this substance will eventually turn into water vapor if exposed to enough sunlight or heat energy from volcanoes (which could help explain why some scientists think there may have been liquid water on Mars).\nWhat would happen if you went to Mars?\nIf you went to Mars, you would be exposed

In [28]:

response2 = "\n\nStephen Colbert and John Oliver, two of the most popular late-night comedians, are facing off in a rap battle to end all rap battles. The stakes are high, with bragging rights on the line and the fate of the world hanging in the balance.\n\nHere's how the rap battle might go down:\n\nColbert:\nYo, I'm the king of the late-night scene,\nMy jokes are so sharp, they'll leave you serene.\nI'm the master of satire, the one they all fear,\nMy wit is so sharp, it'll leave you in tears.\n\nOliver:\nHold up, Stephen, you think you're the best?\nI've got news for you, you're just a mess.\nMy jokes are like a sword, they'll pierce your soul,\nI'm the one they call when they want to take control.\n\nColbert:\nOh really, John? You think you can take me down?\nI've got more Emmy's than you've got frowns.\nMy show's the biggest,"
response = ".\n\nStephen Colbert:  Yo, I heard you tryna come for my crown, John Oliver!\nI'm the king of late night, don't you forget!\nMy jokes are fire, my wit is sharp,\nI'm the one they call when they want to laugh hard!\n\nJohn Oliver:  Oh please, Stephen, you're just a clown,\nYour humor's stale, your jokes are worn out.\nI'm the one who brings the heat, the one who brings the pain,\nMy comedy's like a sword, it's sharp and it's gained!\n\nStephen Colbert:  You may have some clever lines, John, but you can't touch my style,\nMy fans love me, they wouldn't trade me for a while!\nI've been doing this for years, I'm a pro,\nYou're just a Brit who thinks he can flow!\n\nJohn Oliver:  Oh snap, you're going down, Stephen, you're in the zone,\nMy comedy's not just clever, it's a work of art, it's shown"

In [29]:
print(response)

.

Stephen Colbert:  Yo, I heard you tryna come for my crown, John Oliver!
I'm the king of late night, don't you forget!
My jokes are fire, my wit is sharp,
I'm the one they call when they want to laugh hard!

John Oliver:  Oh please, Stephen, you're just a clown,
Your humor's stale, your jokes are worn out.
I'm the one who brings the heat, the one who brings the pain,
My comedy's like a sword, it's sharp and it's gained!

Stephen Colbert:  You may have some clever lines, John, but you can't touch my style,
My fans love me, they wouldn't trade me for a while!
I've been doing this for years, I'm a pro,
You're just a Brit who thinks he can flow!

John Oliver:  Oh snap, you're going down, Stephen, you're in the zone,
My comedy's not just clever, it's a work of art, it's shown
